# STADVDB MCO1: Query Processing in a Data Warehouse
## Research Question: The Impact of Language, Region, and Cultural Adaptation on International Film Reception

**STADVDB_S19 Group 10:**  
- Perol, Francine Nicole T.
- Pineda, Dencel Angelo D.
- Sy, Vaughn Marick A.
- Valdez, Pulvert Gerald M.

In [ ]:
from sqlalchemy import create_engine

DB_USER = "postgres"
DB_PASSWORD = "EleanorKat17;"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mco1_imdb"


# Create the connection engine
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [ ]:
# Test: fetch first 5 rows from your fact table
df = pd.read_sql("SELECT * FROM fact_film_version LIMIT 5;", engine)
df